# Modelo de Suavización Exponencial Doble - Holt (DES)

Este notebook implementa el método de Suavización Exponencial Doble (Double Exponential Smoothing - Holt) para predecir las ventas de dos productos, utilizando tres tipos de validación temporal:
- Walk-Forward Validation
- Rolling Window
- Expanding Window

Posteriormente, se utiliza Optimización Bayesiana para encontrar los mejores hiperparámetros y entrenar el modelo final para producción.


In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings
import optuna
warnings.filterwarnings('ignore')


In [2]:
# Cargar el dataset
df = pd.read_csv('data-set.csv', index_col=0)
print("Dataset cargado:")
print(f"Forma del dataset: {df.shape}")
print(f"\nPrimeras filas:")
print(df.head())
print(f"\nÚltimas filas:")
print(df.tail())
print(f"\nInformación del dataset:")
print(df.info())


Dataset cargado:
Forma del dataset: (127, 2)

Primeras filas:
    producto1   producto2
1  500.000000  200.000000
2  497.400893  210.686220
3  478.605317  222.018584
4  486.454125  233.920990
5  479.695678  238.402098

Últimas filas:
      producto1   producto2
123  164.610771  629.293034
124  150.881839  637.099467
125  151.788470  653.155282
126  137.047639  672.528345
127  141.990873  676.058092

Información del dataset:
<class 'pandas.core.frame.DataFrame'>
Index: 127 entries, 1 to 127
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   producto1  127 non-null    float64
 1   producto2  127 non-null    float64
dtypes: float64(2)
memory usage: 3.0 KB
None


## Función Holt (Double Exponential Smoothing)

Implementación del método de Suavización Exponencial Doble (Holt) para realizar predicciones. En Holt, se utilizan dos parámetros de suavización:
- alpha (α): controla el peso dado a las observaciones más recientes del nivel
- beta (β): controla el peso dado a las observaciones más recientes de la tendencia


In [3]:
def holt_forecast(data, alpha=None, beta=None, forecast_horizon=1):
    """
    Calcula la Suavización Exponencial Doble (Holt) y realiza predicciones.
    
    Parameters:
    -----------
    data : array-like
        Serie temporal de datos
    alpha : float, optional
        Parámetro de suavización del nivel (0 < alpha <= 1). Si es None, se optimiza automáticamente.
    beta : float, optional
        Parámetro de suavización de la tendencia (0 < beta <= 1). Si es None, se optimiza automáticamente.
    forecast_horizon : int
        Número de períodos a predecir (por defecto 1)
    
    Returns:
    --------
    float or array
        Predicción(es) usando Holt
    """
    if len(data) < 2:
        # Si no hay suficientes datos, usar el último valor disponible
        return data[-1] if len(data) > 0 else 0
    
    # Convertir a pandas Series para usar con statsmodels
    data_series = pd.Series(data)
    
    try:
        # Crear y ajustar el modelo Holt
        # Double Exponential Smoothing: trend='add', seasonal=None
        model = ExponentialSmoothing(
            data_series,
            trend='add',
            seasonal=None,
            initialization_method='estimated'
        )
        
        # Si alpha y beta son None, dejar que el modelo los optimice
        # Si están especificados, usarlos
        if alpha is not None and beta is not None:
            fit = model.fit(smoothing_level=alpha, smoothing_trend=beta, optimized=False)
        else:
            fit = model.fit(optimized=True)
        
        # Realizar predicción
        forecast = fit.forecast(steps=forecast_horizon)
        
        # Si solo se predice un paso, devolver un escalar
        if forecast_horizon == 1:
            # Manejar tanto Series como array
            if hasattr(forecast, 'iloc'):
                return float(forecast.iloc[0])
            else:
                return float(forecast[0])
        else:
            # Devolver array de predicciones
            if hasattr(forecast, 'values'):
                return forecast.values
            else:
                return np.array(forecast)
        
    except Exception as e:
        # En caso de error, usar el último valor como predicción
        print(f"Advertencia en Holt: {e}")
        return data[-1]


## Función 1: Walk-Forward Validation
    
En Walk-Forward Validation, se entrena el modelo con datos hasta un punto específico y se valida con el siguiente punto. Luego se avanza un paso y se repite el proceso.


In [4]:
def walk_forward_validation(dataset, alpha=None, beta=None, train_size_min=12):
    """
    Realiza validación Walk-Forward para cada producto del dataset.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset con las columnas de productos
    alpha : float, optional
        Parámetro de suavización del nivel para Holt. Si es None, se optimiza en cada ventana.
    beta : float, optional
        Parámetro de suavización de la tendencia para Holt. Si es None, se optimiza en cada ventana.
    train_size_min : int
        Tamaño mínimo de entrenamiento antes de comenzar la validación (por defecto 12)
    
    Returns:
    --------
    dict
        Diccionario con métricas para cada producto:
        - 'producto1': lista de diccionarios con métricas por iteración
        - 'producto2': lista de diccionarios con métricas por iteración
        Cada diccionario contiene: iteracion, ventana, rmse, mae
    """
    results = {}
    
    for producto in dataset.columns:
        print(f"\n=== Walk-Forward Validation para {producto} ===")
        data = dataset[producto].values
        n = len(data)
        
        metrics_list = []
        
        # Comenzar validación desde train_size_min hasta n-1
        for i in range(train_size_min, n):
            # Datos de entrenamiento: desde el inicio hasta i
            train_data = data[:i]
            # Dato de validación: i
            actual = data[i]
            
            # Realizar predicción
            try:
                prediction = holt_forecast(train_data, alpha=alpha, beta=beta, forecast_horizon=1)
            except Exception as e:
                print(f"Error en iteración {i}: {e}")
                prediction = train_data[-1]  # Fallback al último valor
            
            # Calcular métricas
            rmse = np.sqrt(mean_squared_error([actual], [prediction]))
            mae = mean_absolute_error([actual], [prediction])
            
            metrics_list.append({
                'iteracion': i - train_size_min + 1,
                'ventana': i,
                'rmse': rmse,
                'mae': mae
            })
            
            if (i - train_size_min + 1) % 20 == 0:
                print(f"  Iteración {i - train_size_min + 1}/{n - train_size_min}: RMSE={rmse:.4f}, MAE={mae:.4f}")
        
        results[producto] = metrics_list
        print(f"  Total de iteraciones: {len(metrics_list)}")
    
    return results


## Función 2: Rolling Window Validation

En Rolling Window Validation, se mantiene un tamaño fijo de ventana de entrenamiento que se desplaza a lo largo del tiempo.


In [5]:
def rolling_window_validation(dataset, alpha=None, beta=None, train_window_size=12):
    """
    Realiza validación Rolling Window para cada producto del dataset.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset con las columnas de productos
    alpha : float, optional
        Parámetro de suavización del nivel para Holt. Si es None, se optimiza en cada ventana.
    beta : float, optional
        Parámetro de suavización de la tendencia para Holt. Si es None, se optimiza en cada ventana.
    train_window_size : int
        Tamaño fijo de la ventana de entrenamiento (por defecto 12)
    
    Returns:
    --------
    dict
        Diccionario con métricas para cada producto:
        - 'producto1': lista de diccionarios con métricas por iteración
        - 'producto2': lista de diccionarios con métricas por iteración
        Cada diccionario contiene: iteracion, ventana, rmse, mae
    """
    results = {}
    
    for producto in dataset.columns:
        print(f"\n=== Rolling Window Validation para {producto} ===")
        data = dataset[producto].values
        n = len(data)
        
        metrics_list = []
        
        # Comenzar desde train_window_size hasta n-1
        for i in range(train_window_size, n):
            # Datos de entrenamiento: ventana fija de tamaño train_window_size
            train_data = data[i - train_window_size:i]
            # Dato de validación: i
            actual = data[i]
            
            # Realizar predicción
            try:
                prediction = holt_forecast(train_data, alpha=alpha, beta=beta, forecast_horizon=1)
            except Exception as e:
                print(f"Error en iteración {i}: {e}")
                prediction = train_data[-1]  # Fallback al último valor
            
            # Calcular métricas
            rmse = np.sqrt(mean_squared_error([actual], [prediction]))
            mae = mean_absolute_error([actual], [prediction])
            
            metrics_list.append({
                'iteracion': i - train_window_size + 1,
                'ventana': f"{i - train_window_size}-{i}",
                'rmse': rmse,
                'mae': mae
            })
            
            if (i - train_window_size + 1) % 20 == 0:
                print(f"  Iteración {i - train_window_size + 1}/{n - train_window_size}: RMSE={rmse:.4f}, MAE={mae:.4f}")
        
        results[producto] = metrics_list
        print(f"  Total de iteraciones: {len(metrics_list)}")
    
    return results


## Función 3: Expanding Window Validation

En Expanding Window Validation, la ventana de entrenamiento crece con cada iteración, comenzando desde un tamaño mínimo y expandiéndose hasta incluir todos los datos disponibles hasta ese punto.


In [6]:
def expanding_window_validation(dataset, alpha=None, beta=None, train_size_min=12):
    """
    Realiza validación Expanding Window para cada producto del dataset.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset con las columnas de productos
    alpha : float, optional
        Parámetro de suavización del nivel para Holt. Si es None, se optimiza en cada ventana.
    beta : float, optional
        Parámetro de suavización de la tendencia para Holt. Si es None, se optimiza en cada ventana.
    train_size_min : int
        Tamaño mínimo inicial de la ventana de entrenamiento (por defecto 12)
    
    Returns:
    --------
    dict
        Diccionario con métricas para cada producto:
        - 'producto1': lista de diccionarios con métricas por iteración
        - 'producto2': lista de diccionarios con métricas por iteración
        Cada diccionario contiene: iteracion, ventana, rmse, mae
    """
    results = {}
    
    for producto in dataset.columns:
        print(f"\n=== Expanding Window Validation para {producto} ===")
        data = dataset[producto].values
        n = len(data)
        
        metrics_list = []
        
        # Comenzar validación desde train_size_min hasta n-1
        for i in range(train_size_min, n):
            # Datos de entrenamiento: desde el inicio hasta i (ventana que se expande)
            train_data = data[:i]
            # Dato de validación: i
            actual = data[i]
            
            # Realizar predicción
            try:
                prediction = holt_forecast(train_data, alpha=alpha, beta=beta, forecast_horizon=1)
            except Exception as e:
                print(f"Error en iteración {i}: {e}")
                prediction = train_data[-1]  # Fallback al último valor
            
            # Calcular métricas
            rmse = np.sqrt(mean_squared_error([actual], [prediction]))
            mae = mean_absolute_error([actual], [prediction])
            
            metrics_list.append({
                'iteracion': i - train_size_min + 1,
                'ventana': i,  # Tamaño de la ventana que se expande
                'rmse': rmse,
                'mae': mae
            })
            
            if (i - train_size_min + 1) % 20 == 0:
                print(f"  Iteración {i - train_size_min + 1}/{n - train_size_min}: RMSE={rmse:.4f}, MAE={mae:.4f}")
        
        results[producto] = metrics_list
        print(f"  Total de iteraciones: {len(metrics_list)}")
    
    return results


## Ejecución de las Validaciones

Después de definir las funciones que permitirán ejecutar el modelo teniendo en cuenta los tres tipos de validación, ahora ejecutamos las tres validaciones para cada producto y calculamos las métricas promedio.


In [7]:
# Parámetros de validación
TRAIN_SIZE_MIN = 12  # Tamaño mínimo de entrenamiento
TRAIN_WINDOW_SIZE = 12  # Tamaño fijo para Rolling Window

# Ejecutar Walk-Forward Validation
print("=" * 60)
print("WALK-FORWARD VALIDATION")
print("=" * 60)
results_wf = walk_forward_validation(df, alpha=None, beta=None, train_size_min=TRAIN_SIZE_MIN)


WALK-FORWARD VALIDATION

=== Walk-Forward Validation para producto1 ===
  Iteración 20/115: RMSE=1.4133, MAE=1.4133
  Iteración 40/115: RMSE=6.7388, MAE=6.7388
  Iteración 60/115: RMSE=8.9019, MAE=8.9019
  Iteración 80/115: RMSE=8.2118, MAE=8.2118
  Iteración 100/115: RMSE=15.1311, MAE=15.1311
  Total de iteraciones: 115

=== Walk-Forward Validation para producto2 ===
  Iteración 20/115: RMSE=12.9458, MAE=12.9458
  Iteración 40/115: RMSE=7.4010, MAE=7.4010
  Iteración 60/115: RMSE=1.4761, MAE=1.4761
  Iteración 80/115: RMSE=26.1809, MAE=26.1809
  Iteración 100/115: RMSE=8.3415, MAE=8.3415
  Total de iteraciones: 115


In [8]:
# Ejecutar Rolling Window Validation
print("\n" + "=" * 60)
print("ROLLING WINDOW VALIDATION")
print("=" * 60)
results_rw = rolling_window_validation(df, alpha=None, beta=None, train_window_size=TRAIN_WINDOW_SIZE)



ROLLING WINDOW VALIDATION

=== Rolling Window Validation para producto1 ===
  Iteración 20/115: RMSE=1.6922, MAE=1.6922
  Iteración 40/115: RMSE=2.7692, MAE=2.7692
  Iteración 60/115: RMSE=1.5471, MAE=1.5471
  Iteración 80/115: RMSE=14.3658, MAE=14.3658
  Iteración 100/115: RMSE=10.0089, MAE=10.0089
  Total de iteraciones: 115

=== Rolling Window Validation para producto2 ===
  Iteración 20/115: RMSE=14.8344, MAE=14.8344
  Iteración 40/115: RMSE=9.8621, MAE=9.8621
  Iteración 60/115: RMSE=8.6249, MAE=8.6249
  Iteración 80/115: RMSE=38.8432, MAE=38.8432
  Iteración 100/115: RMSE=10.5202, MAE=10.5202
  Total de iteraciones: 115


In [9]:
# Ejecutar Expanding Window Validation
print("\n" + "=" * 60)
print("EXPANDING WINDOW VALIDATION")
print("=" * 60)
results_ew = expanding_window_validation(df, alpha=None, beta=None, train_size_min=TRAIN_SIZE_MIN)



EXPANDING WINDOW VALIDATION

=== Expanding Window Validation para producto1 ===
  Iteración 20/115: RMSE=1.4133, MAE=1.4133
  Iteración 40/115: RMSE=6.7388, MAE=6.7388
  Iteración 60/115: RMSE=8.9019, MAE=8.9019
  Iteración 80/115: RMSE=8.2118, MAE=8.2118
  Iteración 100/115: RMSE=15.1311, MAE=15.1311
  Total de iteraciones: 115

=== Expanding Window Validation para producto2 ===
  Iteración 20/115: RMSE=12.9458, MAE=12.9458
  Iteración 40/115: RMSE=7.4010, MAE=7.4010
  Iteración 60/115: RMSE=1.4761, MAE=1.4761
  Iteración 80/115: RMSE=26.1809, MAE=26.1809
  Iteración 100/115: RMSE=8.3415, MAE=8.3415
  Total de iteraciones: 115


In [10]:
def calcular_metricas_promedio(results, nombre_validacion):
    """
    Calcula las métricas promedio para cada producto.
    
    Parameters:
    -----------
    results : dict
        Resultados de la validación
    nombre_validacion : str
        Nombre del tipo de validación
    
    Returns:
    --------
    dict
        Diccionario con métricas promedio por producto
    """
    metricas_promedio = {}
    
    print(f"\n{'='*60}")
    print(f"MÉTRICAS PROMEDIO - {nombre_validacion.upper()}")
    print(f"{'='*60}")
    
    for producto in results.keys():
        metrics_list = results[producto]
        
        # Calcular promedios
        rmse_promedio = np.mean([m['rmse'] for m in metrics_list])
        mae_promedio = np.mean([m['mae'] for m in metrics_list])
        
        metricas_promedio[producto] = {
            'rmse_promedio': rmse_promedio,
            'mae_promedio': mae_promedio,
            'num_iteraciones': len(metrics_list)
        }
        
        print(f"\n{producto}:")
        print(f"  RMSE Promedio: {rmse_promedio:.4f}")
        print(f"  MAE Promedio: {mae_promedio:.4f}")
        print(f"  Número de iteraciones: {len(metrics_list)}")
    
    return metricas_promedio

# Calcular métricas promedio para cada tipo de validación
metricas_wf = calcular_metricas_promedio(results_wf, "Walk-Forward")
metricas_rw = calcular_metricas_promedio(results_rw, "Rolling Window")
metricas_ew = calcular_metricas_promedio(results_ew, "Expanding Window")



MÉTRICAS PROMEDIO - WALK-FORWARD

producto1:
  RMSE Promedio: 7.0768
  MAE Promedio: 7.0768
  Número de iteraciones: 115

producto2:
  RMSE Promedio: 12.2762
  MAE Promedio: 12.2762
  Número de iteraciones: 115

MÉTRICAS PROMEDIO - ROLLING WINDOW

producto1:
  RMSE Promedio: 7.3847
  MAE Promedio: 7.3847
  Número de iteraciones: 115

producto2:
  RMSE Promedio: 12.4054
  MAE Promedio: 12.4054
  Número de iteraciones: 115

MÉTRICAS PROMEDIO - EXPANDING WINDOW

producto1:
  RMSE Promedio: 7.0768
  MAE Promedio: 7.0768
  Número de iteraciones: 115

producto2:
  RMSE Promedio: 12.2762
  MAE Promedio: 12.2762
  Número de iteraciones: 115


## Comparación de Resultados y Selección del Mejor Método de Validación

Comparamos los RMSE promedio de cada tipo de validación para seleccionar el mejor método.


In [11]:
# Crear DataFrame comparativo
comparacion = []

for producto in df.columns:
    comparacion.append({
        'Producto': producto,
        'Validacion': 'Walk-Forward',
        'RMSE_Promedio': metricas_wf[producto]['rmse_promedio'],
        'MAE_Promedio': metricas_wf[producto]['mae_promedio']
    })
    comparacion.append({
        'Producto': producto,
        'Validacion': 'Rolling Window',
        'RMSE_Promedio': metricas_rw[producto]['rmse_promedio'],
        'MAE_Promedio': metricas_rw[producto]['mae_promedio']
    })
    comparacion.append({
        'Producto': producto,
        'Validacion': 'Expanding Window',
        'RMSE_Promedio': metricas_ew[producto]['rmse_promedio'],
        'MAE_Promedio': metricas_ew[producto]['mae_promedio']
    })

df_comparacion = pd.DataFrame(comparacion)
print("\n" + "=" * 80)
print("COMPARACIÓN DE MÉTODOS DE VALIDACIÓN")
print("=" * 80)
print(df_comparacion.to_string(index=False))

# Identificar el mejor método para cada producto
print("\n" + "=" * 80)
print("MEJOR MÉTODO DE VALIDACIÓN POR PRODUCTO (basado en RMSE Promedio)")
print("=" * 80)
for producto in df.columns:
    producto_df = df_comparacion[df_comparacion['Producto'] == producto]
    mejor = producto_df.loc[producto_df['RMSE_Promedio'].idxmin()]
    print(f"\n{producto}:")
    print(f"  Mejor método: {mejor['Validacion']}")
    print(f"  RMSE Promedio: {mejor['RMSE_Promedio']:.4f}")



COMPARACIÓN DE MÉTODOS DE VALIDACIÓN
 Producto       Validacion  RMSE_Promedio  MAE_Promedio
producto1     Walk-Forward       7.076816      7.076816
producto1   Rolling Window       7.384706      7.384706
producto1 Expanding Window       7.076814      7.076814
producto2     Walk-Forward      12.276188     12.276188
producto2   Rolling Window      12.405437     12.405437
producto2 Expanding Window      12.276187     12.276187

MEJOR MÉTODO DE VALIDACIÓN POR PRODUCTO (basado en RMSE Promedio)

producto1:
  Mejor método: Expanding Window
  RMSE Promedio: 7.0768

producto2:
  Mejor método: Expanding Window
  RMSE Promedio: 12.2762


## Optimización Bayesiana para Encontrar los Mejores Parámetros (Alpha y Beta)

Utilizamos Optimización Bayesiana (Optuna) para encontrar los mejores parámetros alpha y beta para cada producto, usando el mejor método de validación identificado.


In [12]:
def objective_walk_forward(trial, data, train_size_min=12):
    """
    Función objetivo para optimización bayesiana usando Walk-Forward Validation.
    """
    alpha = trial.suggest_float('alpha', 0.01, 0.99, log=False)
    beta = trial.suggest_float('beta', 0.01, 0.99, log=False)
    
    n = len(data)
    errors = []
    
    for i in range(train_size_min, n):
        train_data = data[:i]
        actual = data[i]
        
        try:
            prediction = holt_forecast(train_data, alpha=alpha, beta=beta, forecast_horizon=1)
            error = np.sqrt(mean_squared_error([actual], [prediction]))
            if np.isfinite(error):
                errors.append(error)
        except:
            continue
    
    if len(errors) == 0:
        return float('inf')
    
    return np.mean(errors)

def objective_rolling_window(trial, data, train_window_size=12):
    """
    Función objetivo para optimización bayesiana usando Rolling Window Validation.
    """
    alpha = trial.suggest_float('alpha', 0.01, 0.99, log=False)
    beta = trial.suggest_float('beta', 0.01, 0.99, log=False)
    
    n = len(data)
    errors = []
    
    for i in range(train_window_size, n):
        train_data = data[i - train_window_size:i]
        actual = data[i]
        
        try:
            prediction = holt_forecast(train_data, alpha=alpha, beta=beta, forecast_horizon=1)
            error = np.sqrt(mean_squared_error([actual], [prediction]))
            if np.isfinite(error):
                errors.append(error)
        except:
            continue
    
    if len(errors) == 0:
        return float('inf')
    
    return np.mean(errors)

def objective_expanding_window(trial, data, train_size_min=12):
    """
    Función objetivo para optimización bayesiana usando Expanding Window Validation.
    """
    alpha = trial.suggest_float('alpha', 0.01, 0.99, log=False)
    beta = trial.suggest_float('beta', 0.01, 0.99, log=False)
    
    n = len(data)
    errors = []
    
    for i in range(train_size_min, n):
        train_data = data[:i]
        actual = data[i]
        
        try:
            prediction = holt_forecast(train_data, alpha=alpha, beta=beta, forecast_horizon=1)
            error = np.sqrt(mean_squared_error([actual], [prediction]))
            if np.isfinite(error):
                errors.append(error)
        except:
            continue
    
    if len(errors) == 0:
        return float('inf')
    
    return np.mean(errors)


In [13]:
# Determinar el mejor método de validación para cada producto
mejor_metodo_por_producto = {}

for producto in df.columns:
    producto_df = df_comparacion[df_comparacion['Producto'] == producto]
    mejor_idx = producto_df['RMSE_Promedio'].idxmin()
    mejor_metodo = producto_df.loc[mejor_idx, 'Validacion']
    mejor_metodo_por_producto[producto] = mejor_metodo
    print(f"{producto}: Mejor método = {mejor_metodo}")

# Realizar optimización bayesiana para cada producto usando su mejor método
mejores_params = {}

print("\n" + "=" * 80)
print("OPTIMIZACIÓN BAYESIANA")
print("=" * 80)

for producto in df.columns:
    print(f"\n--- Optimizando {producto} ---")
    data = df[producto].values
    mejor_metodo = mejor_metodo_por_producto[producto]
    
    # Crear función objetivo según el mejor método
    if mejor_metodo == 'Walk-Forward':
        objective_func = lambda trial: objective_walk_forward(trial, data, TRAIN_SIZE_MIN)
    elif mejor_metodo == 'Rolling Window':
        objective_func = lambda trial: objective_rolling_window(trial, data, TRAIN_WINDOW_SIZE)
    else:  # Expanding Window
        objective_func = lambda trial: objective_expanding_window(trial, data, TRAIN_SIZE_MIN)
    
    # Crear estudio de Optuna
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler())
    
    # Ejecutar optimización
    study.optimize(objective_func, n_trials=50, show_progress_bar=True)
    
    mejores_params[producto] = {
        'alpha': study.best_params['alpha'],
        'beta': study.best_params['beta'],
        'best_rmse': study.best_value,
        'metodo_validacion': mejor_metodo
    }
    
    print(f"  Mejor alpha: {study.best_params['alpha']:.4f}")
    print(f"  Mejor beta: {study.best_params['beta']:.4f}")
    print(f"  Mejor RMSE: {study.best_value:.4f}")
    print(f"  Método usado: {mejor_metodo}")

print("\n" + "=" * 80)
print("RESUMEN DE MEJORES PARÁMETROS")
print("=" * 80)
for producto, params in mejores_params.items():
    print(f"\n{producto}:")
    print(f"  Alpha óptimo: {params['alpha']:.4f}")
    print(f"  Beta óptimo: {params['beta']:.4f}")
    print(f"  RMSE: {params['best_rmse']:.4f}")
    print(f"  Método de validación: {params['metodo_validacion']}")


[I 2025-11-30 13:13:14,826] A new study created in memory with name: no-name-8e343e22-3f43-4e81-8c66-bc1e2c892091


producto1: Mejor método = Expanding Window
producto2: Mejor método = Expanding Window

OPTIMIZACIÓN BAYESIANA

--- Optimizando producto1 ---


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-11-30 13:13:15,118] Trial 0 finished with value: 6.581963061261775 and parameters: {'alpha': 0.7489757122687674, 'beta': 0.40588177962994315}. Best is trial 0 with value: 6.581963061261775.
[I 2025-11-30 13:13:15,398] Trial 1 finished with value: 6.961734083681842 and parameters: {'alpha': 0.5505044242468033, 'beta': 0.4113632679611899}. Best is trial 0 with value: 6.581963061261775.
[I 2025-11-30 13:13:15,965] Trial 2 finished with value: 26.05371877200114 and parameters: {'alpha': 0.1378265562280264, 'beta': 0.16116481576294633}. Best is trial 0 with value: 6.581963061261775.
[I 2025-11-30 13:13:16,286] Trial 3 finished with value: 6.987057956650726 and parameters: {'alpha': 0.8422398393727549, 'beta': 0.5614940043492134}. Best is trial 0 with value: 6.581963061261775.
[I 2025-11-30 13:13:16,606] Trial 4 finished with value: 7.156981301921848 and parameters: {'alpha': 0.97337858687094, 'beta': 0.1909790748243071}. Best is trial 0 with value: 6.581963061261775.
[I 2025-11-30 1

[I 2025-11-30 13:13:28,025] A new study created in memory with name: no-name-279c602b-6d00-4278-abfb-534c88fe90f2


[I 2025-11-30 13:13:27,786] Trial 48 finished with value: 6.377868695835995 and parameters: {'alpha': 0.4964985839123986, 'beta': 0.826283933025832}. Best is trial 33 with value: 6.3428099395184985.
[I 2025-11-30 13:13:28,022] Trial 49 finished with value: 6.3890959210965645 and parameters: {'alpha': 0.5608977949947651, 'beta': 0.8724512751167}. Best is trial 33 with value: 6.3428099395184985.
  Mejor alpha: 0.5282
  Mejor beta: 0.8109
  Mejor RMSE: 6.3428
  Método usado: Expanding Window

--- Optimizando producto2 ---


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-11-30 13:13:28,269] Trial 0 finished with value: 21.677645188437022 and parameters: {'alpha': 0.2317436787870574, 'beta': 0.14470750953279854}. Best is trial 0 with value: 21.677645188437022.
[I 2025-11-30 13:13:28,508] Trial 1 finished with value: 13.190621655692796 and parameters: {'alpha': 0.7560134566173258, 'beta': 0.9174157735436176}. Best is trial 1 with value: 13.190621655692796.
[I 2025-11-30 13:13:28,743] Trial 2 finished with value: 58.95345361139687 and parameters: {'alpha': 0.03329954885647253, 'beta': 0.056302211559730884}. Best is trial 1 with value: 13.190621655692796.
[I 2025-11-30 13:13:29,044] Trial 3 finished with value: 30.863915795956565 and parameters: {'alpha': 0.0741459420455869, 'beta': 0.11141830189019968}. Best is trial 1 with value: 13.190621655692796.
[I 2025-11-30 13:13:29,333] Trial 4 finished with value: 31.05536392930096 and parameters: {'alpha': 0.0703845903769824, 'beta': 0.12969735642545738}. Best is trial 1 with value: 13.190621655692796.
[

## Modelo Final para Producción

Entrenamos el modelo final con los mejores parámetros encontrados usando todos los datos disponibles y realizamos predicciones para producción (1 pronóstico para el siguiente mes).


In [14]:
# Entrenar modelos finales para cada producto con los mejores parámetros
modelos_finales = {}
predicciones_finales = {}

print("=" * 80)
print("ENTRENAMIENTO DE MODELOS FINALES PARA PRODUCCIÓN")
print("=" * 80)

for producto in df.columns:
    print(f"\n--- {producto} ---")
    data = df[producto].values
    alpha_optimo = mejores_params[producto]['alpha']
    beta_optimo = mejores_params[producto]['beta']
    
    # Entrenar modelo final con todos los datos
    data_series = pd.Series(data)
    model = ExponentialSmoothing(
        data_series,
        trend='add',
        seasonal=None,
        initialization_method='estimated'
    )
    fit = model.fit(smoothing_level=alpha_optimo, smoothing_trend=beta_optimo, optimized=False)
    
    modelos_finales[producto] = fit
    
    # Realizar predicción para el próximo mes (1 solo pronóstico)
    forecast_horizon = 1
    forecast = fit.forecast(steps=forecast_horizon)
    
    # Extraer el valor de la predicción (manejar tanto Series como array)
    if hasattr(forecast, 'iloc'):
        forecast_value = float(forecast.iloc[0])
    elif hasattr(forecast, 'values'):
        forecast_value = float(forecast.values[0])
    else:
        forecast_value = float(forecast[0])
    
    predicciones_finales[producto] = forecast_value
    
    print(f"  Alpha usado: {alpha_optimo:.4f}")
    print(f"  Beta usado: {beta_optimo:.4f}")
    print(f"  Último valor observado: {data[-1]:.4f}")
    print(f"  Predicción para el próximo mes: {forecast_value:.4f}")

# Crear DataFrame con las predicciones
df_predicciones = pd.DataFrame([predicciones_finales])
df_predicciones.index = ['Siguiente_Mes']

print("\n" + "=" * 80)
print("PREDICCIONES FINALES PARA PRODUCCIÓN")
print("=" * 80)
print(df_predicciones)


ENTRENAMIENTO DE MODELOS FINALES PARA PRODUCCIÓN

--- producto1 ---
  Alpha usado: 0.5282
  Beta usado: 0.8109
  Último valor observado: 141.9909
  Predicción para el próximo mes: 132.3972

--- producto2 ---
  Alpha usado: 0.4928
  Beta usado: 0.8708
  Último valor observado: 676.0581
  Predicción para el próximo mes: 692.2846

PREDICCIONES FINALES PARA PRODUCCIÓN
                producto1   producto2
Siguiente_Mes  132.397207  692.284626


## Resumen Final

### Resultados de Validación

- **Walk-Forward Validation**: Se evaluó el modelo avanzando paso a paso en el tiempo
- **Rolling Window Validation**: Se evaluó con una ventana fija que se desplaza
- **Expanding Window Validation**: Se evaluó con una ventana que crece progresivamente

### Optimización Bayesiana

Se utilizó Optimización Bayesiana (Optuna) para encontrar los mejores parámetros alpha y beta para cada producto, usando el método de validación que obtuvo el menor RMSE promedio.

### Modelo Final

Los modelos finales fueron entrenados con todos los datos disponibles usando los mejores parámetros encontrados (alpha y beta), y están listos para realizar predicciones en producción para el siguiente mes.
